In [ ]:
from pathlib import Path
root=Path("dataset")
for split in ("train","val","test"):
    imgs=sorted((root/"images"/split).glob("*.*"))
    miss=[p for p in imgs if not (root/"labels"/split/f"{p.stem}.txt").exists()]
    print(split, "images:", len(imgs), "missing_labels:", len(miss))
    if miss[:3]: print(" examples:", [m.name for m in miss[:3]])

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from ultralytics import YOLO
model = YOLO("yolo11n-seg.pt")  # n s m l x (smallest to largest)

model.train(
    data="data.yaml",
    imgsz=512,
    batch=1,
    amp=True,
    mosaic=0.0,
    rect=True,
    workers=1,
    device=0,
    patience=25,
    epochs=10
)


In [ ]:
model.val(data="data.yaml")

In [ ]:
model.predict(source="dataset/images/val", imgsz=896, conf=0.25, save=True)

In [ ]:
model.export(format="onnx")         # or "engine" for TensorRT, "torchscript", etc.